In [10]:
import math
import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.basic import Expr
from docplex.mp.model import Model
from docplex.mp.solution import SolveSolution
from docplex.util.status import JobSolveStatus
import cplex.callbacks as cpx_cb
from Functions_validation.braking_poly import p as Pbraking
from Functions_validation.traction_poly import p as Ptraction
from Functions_validation.RAV_poly import P as Prav
from Functions_validation.braking_poly import Pdecc as Pdecc
from Functions_validation.traction_poly import Pacc as Pacc
from Functions_validation.power_poly import P as Ppower

In [11]:
# Define some constants
A, B, C, m, v_max, p_max, c_max = 0.632, 40.7, 3900, 320000, 220, 4305220, 320000


## Defining the Model

### Define model

In [2]:
opt_model = Model(name="Poly_Model")
N = 20
Dist = 30
tolerance = 0.005

### Define variables

In [3]:
v={}
for i in range(0, N):
    v[i]= opt_model.continuous_var(name=f"v_{i}")

d={}
for i in range(0, N):
    d[i]= opt_model.continuous_var(name=f"d_{i}")

kroe={}
for i in range(0, N):
    kroe[i]= opt_model.binary_var(name=f"kroe_{i}")

### Define cost

In [4]:
# Total cost
total_cost = opt_model.sum(Ppower(v[i], d[i]) for i in range(N))

# Set objective
opt_model.minimize(total_cost)

### Kroe constraints

In [5]:
M = 1
epsilon = 1e-4

# Constraint: if delta == 1, then d >= epsilon
for i in range(0, N):
    opt_model.add_constraint(d[i] >= epsilon - M * (1 - kroe[i]))

    # Constraint: if delta == 0, then d <= 0
    opt_model.add_constraint(d[i] <= M * kroe[i])

### Speed constraints

In [23]:
opt_model.add_constraint(v[0] == 0, "Initial velocity")
opt_model.add_constraint(v[N-1] == 0, "Final velocity")

for i in range(1,N-2):
    opt_model.add_constraint(v[i+1] - v[i] + (1/m)*Prav(v[i]) == (1/m)(d[i]*(kroe[i]*Ptraction(v[i]) + (1-kroe[i])*Pbraking(v[i]))), f"Velocity constraint {i}")

DOcplexException: Cannot multiply d_1 by -421.526v_1*kroe_1+-628.475v_1+41582.014kroe_1+213445.587, some terms would have degree >= 3. Maximum polynomial degree is 2.